In [ ]:
import requests
from lxml import etree

# Museum-Digitals OAI-Schnittstelle

Museum-Digital hat nun auch eine [OAI-Schnittstelle](https://www.openarchives.org/pmh/) (vgl. [Blog-Beitrag vom 24.11.2025](https://blog.museum-digital.org/2025/11/24/making-interoperability-easy/)) für den bequemen Abzug größerer Datenmengen.

Hier ein paar praktische Anwendungsbeispiele.

## Abzug der Metadaten der Kurt-Schwarz-Sammlung

Die Sammlungs-ID auf der Berliner-Instanz lautet `783` (vgl. https://berlin.museum-digital.de/collection/783).

Folgende URL liefert die ersten 40 Datensätze der Sammlung 783 im LIDO-Format: `https://berlin.museum-digital.de/oai/collection:783?verb=ListRecords&metadataPrefix=lido`

In [ ]:
requests.get("https://berlin.museum-digital.de/oai/collection:783?verb=ListRecords&metadataPrefix=lido").content

Um alle Datensätze abzurufen, muss man einen Blick in das `<resumptionToken>`-Element werfen. Es enhält ein `completeListSize`-Attribut, das den Gesamtumfang der Ergebnismenge angibt, sowie einen Element-Text, der in diesem Fall als JSON-Objekt realisiert ist. Dieses enthält die Informationen, die die OAI-Schnittstelle benötigt, um die weitere Tranche der Ergebnismenge auszugeben.

In [ ]:
res = requests.get("https://berlin.museum-digital.de/oai/collection:783?verb=ListRecords&metadataPrefix=lido")
tree = etree.fromstring(res.content)
NSMAP = { 'oai' : 'http://www.openarchives.org/OAI/2.0/' }
resumptionToken = tree.xpath("oai:ListRecords/oai:resumptionToken", namespaces = NSMAP)

In [ ]:
print("Attribut: ", resumptionToken[0].attrib)

resToken = resumptionToken[0].text
print("Token: ", resToken)

Bei der nächsten Abfrage wird das `resumptionToken` als Attribut übergeben (die Spezifizierung des Ausgabeformats ist nicht mehr nötig), also:

In [ ]:
res2 = requests.get(f"https://berlin.museum-digital.de/oai/collection:783?verb=ListRecords&resumptionToken={resToken}")
res2.content

Dieses Spiel wird wiederholt, bis eine Antwort kein `resmptionToken` mehr enthält.

In eine Schleife gepackt könnte das so aussehen:

In [ ]:
NSMAP = {
    'oai' : 'http://www.openarchives.org/OAI/2.0/',
    'lido' : 'http://www.lido-schema.org'
}

lidos = list()
url = "https://berlin.museum-digital.de/oai/collection:783"
params = {
        'verb' : 'ListRecords',
        'metadataPrefix' : 'lido'
    }
while True:
    res = requests.get(url, params = params)
    tree = etree.fromstring(res.content)
    lidos.extend(tree.xpath('//lido:lido', namespaces = NSMAP))
    resumptionToken = tree.xpath("oai:ListRecords/oai:resumptionToken", namespaces = NSMAP)
    if len(resumptionToken) == 0:
        break
    else:
        params['resumptionToken'] = resumptionToken[0].text
        print(resumptionToken[0].text)

Die LIDO-Datensätze wurden jetzt einfach in einer Liste abgelegt. Sehen wir uns z.B. die `lidoRecID`s an!

In [ ]:
for l in lidos:
    print(l.find('lido:lidoRecID', NSMAP).text)